In [1]:
import numpy as np 
import pandas as pd 

# For visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# For apriori and association rules
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
# Load dataset
df = pd.read_csv('Retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,01/12/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,01/12/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,01/12/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,01/12/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,01/12/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
df.shape

(541909, 8)

In [4]:
if df['InvoiceNo'].str.contains('C|[^0-9]+').any():
    print("There are 'C' or string values in the InvoiceNo column")
else:
    print("There are no 'C' or string values in the InvoiceNo column")

There are 'C' or string values in the InvoiceNo column


In [5]:
# check for 'C' in InvoiceNo column
contains_c = df['InvoiceNo'].str.contains('C')
print(contains_c.any()) # True if there's at least one 'C' in InvoiceNo column

# check for non-numeric characters in InvoiceNo column
non_numeric = df['InvoiceNo'].str.contains('[^0-9]+')
print(non_numeric.any()) # True if there's at least one non-numeric character in InvoiceNo column

True
True


In [6]:
# Data Preprocessing 
#1.) Removing nulls and whitespaces 

if df.isnull().values.any():
    print("Dataframe contains null values")
else:
    print("Dataframe does not contain null values")

Dataframe contains null values


In [7]:
# Check for duplicates in the entire dataframe
if df.duplicated().any():
    print("Dataframe contains duplicates")
else:
    print("Dataframe does not contain duplicates")

Dataframe contains duplicates


In [8]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace = True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('B|[^0-9]+')]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,01/12/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,01/12/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,01/12/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,01/12/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,01/12/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12.0,09/12/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6.0,09/12/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4.0,09/12/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4.0,09/12/2011 12:50,4.15,12680.0,France


In [9]:
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12.0,09/12/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6.0,09/12/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4.0,09/12/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4.0,09/12/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3.0,09/12/2011 12:50,4.95,12680.0,France


In [10]:
basket = (df[df['Country'] == "United Kingdom"]
         .groupby(['InvoiceNo', 'Description'])['Quantity']
         .sum().unstack().reset_index().fillna(0)
         .set_index('InvoiceNo'))
basket

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,label mix up,mystery! Only ever imported 1800,on cargo order,printing smudges/thrown away,rcvd be air temp fix for dotcom sit,re dotcom quick fix.,re-adjustment,samples,samples/damages,smashed
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
item_counts = df[df['Country'] == 'United Kingdom'].groupby('Description')['InvoiceNo'].nunique().reset_index()
item_counts.columns = ['Description', 'Count']
item_counts = item_counts.sort_values('Count', ascending=False).reset_index(drop=True)


In [12]:
item_counts.head(10).style.background_gradient(cmap='Reds')

,Description,Count
0,WHITE HANGING HEART T-LIGHT HOLDER,2166
1,JUMBO BAG RED RETROSPOT,1938
2,REGENCY CAKESTAND 3 TIER,1685
3,PARTY BUNTING,1594
4,LUNCH BAG RED RETROSPOT,1392
5,ASSORTED COLOUR BIRD ORNAMENT,1371
6,SET OF 3 CAKE TINS PANTRY DESIGN,1241
7,NATURAL SLATE HEART CHALKBOARD,1219
8,LUNCH BAG BLACK SKULL.,1216
9,HEART OF WICKER SMALL,1164


In [13]:
def encode_units(x):
    if x <=0:
       return 0
    if x >= 1:
       return 1
basket_sets = basket.applymap(encode_units)
basket_sets.drop(basket_sets.filter(like='wrong' and 'wet').columns, axis=1, inplace=True)
basket_sets = basket_sets.astype(bool)
basket_sets

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,label mix up,mystery! Only ever imported 1800,on cargo order,printing smudges/thrown away,rcvd be air temp fix for dotcom sit,re dotcom quick fix.,re-adjustment,samples,samples/damages,smashed
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536366,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536367,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536368,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536369,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581583,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581584,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
frequent_itemsets = apriori(basket_sets, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric ='lift', min_threshold=.5)
print(rules.head())


                         antecedents                        consequents  \
0      (60 TEATIME FAIRY CAKE CASES)  (PACK OF 72 RETROSPOT CAKE CASES)   
1  (PACK OF 72 RETROSPOT CAKE CASES)      (60 TEATIME FAIRY CAKE CASES)   
2         (ALARM CLOCK BAKELIKE RED)       (ALARM CLOCK BAKELIKE GREEN)   
3       (ALARM CLOCK BAKELIKE GREEN)         (ALARM CLOCK BAKELIKE RED)   
4         (ALARM CLOCK BAKELIKE RED)        (ALARM CLOCK BAKELIKE PINK)   

   antecedent support  consequent support   support  confidence       lift  \
0            0.041789            0.062656  0.022698    0.543161   8.668922   
1            0.062656            0.041789  0.022698    0.362267   8.668922   
2            0.051612            0.048615  0.031245    0.605376  12.452370   
3            0.048615            0.051612  0.031245    0.642694  12.452370   
4            0.051612            0.036406  0.021810    0.422581  11.607440   

   leverage  conviction  zhangs_metric  
0  0.020080    2.051802       0.923226 

In [15]:
rules[(rules['lift']>=6) &
(rules['confidence'] >=0.8)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
25,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.039014,0.051834,0.032022,0.820768,15.834498,0.029999,5.290163,0.974881
187,"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.032022,0.053111,0.027360,0.854419,16.087548,0.025659,6.504228,0.968865
188,"(ROSES REGENCY TEACUP AND SAUCER, PINK REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.030301,0.051834,0.027360,0.902930,17.419596,0.025789,9.767897,0.972048
192,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ...",(JUMBO BAG RED RETROSPOT),0.025196,0.107553,0.020312,0.806167,7.495527,0.017602,4.604214,0.888986
198,"(JUMBO STORAGE BAG SUKI, JUMBO BAG PINK POLKADOT)",(JUMBO BAG RED RETROSPOT),0.028026,0.107553,0.022476,0.801980,7.456595,0.019462,4.506857,0.890858


In [16]:
rules[(rules['lift']>=5) &
(rules['confidence'] >=0.7)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
8,(CHARLOTTE BAG PINK POLKADOT),(RED RETROSPOT CHARLOTTE BAG),0.037405,0.050225,0.026583,0.710682,14.150042,0.024704,3.282813,0.965441
23,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.041623,0.049836,0.030024,0.721333,14.474059,0.027950,3.409678,0.971341
25,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.039014,0.051834,0.032022,0.820768,15.834498,0.029999,5.290163,0.974881
28,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.051834,0.053111,0.038903,0.750535,14.131553,0.036150,3.795685,0.980036
29,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.053111,0.051834,0.038903,0.732497,14.131553,0.036150,3.544511,0.981357
165,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.039014,0.053111,0.030301,0.776671,14.623659,0.028229,4.239893,0.969440
184,(WOODEN HEART CHRISTMAS SCANDINAVIAN),(WOODEN STAR CHRISTMAS SCANDINAVIAN),0.028248,0.026583,0.020423,0.722986,27.197263,0.019672,3.513966,0.991232
185,(WOODEN STAR CHRISTMAS SCANDINAVIAN),(WOODEN HEART CHRISTMAS SCANDINAVIAN),0.026583,0.028248,0.020423,0.768267,27.197263,0.019672,4.193416,0.989536
186,"(GREEN REGENCY TEACUP AND SAUCER, ROSES REGENC...",(PINK REGENCY TEACUP AND SAUCER),0.038903,0.039014,0.027360,0.703281,18.026203,0.025842,3.238706,0.982758
187,"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.032022,0.053111,0.027360,0.854419,16.087548,0.025659,6.504228,0.968865
